In [1]:
import pandas as pd 
import numpy as np 
import pickle 

In [2]:
import numpy as np
import pandas as pd
import re
# from lxml import etree
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import os
import pickle

import konlpy
import jpype
from konlpy.tag import Okt
from konlpy.tag import Komoran
from konlpy.tag import Twitter
from gensim.models.word2vec import Word2Vec
from gensim.models.doc2vec import *
from gensim.models import KeyedVectors
from gensim.models import FastText

In [15]:
finalresult = file_path+"finalresult/result%s/result_%s.csv"%(num,name)
finalresult

'C:/Users/Administrator/Anaconda3/envs/temp/EmotionEmbedding/emotion8_finalcode/finalresult/result3/result_pe8_3.csv'

In [16]:
name = 'pe8_3'
num = name[4:]

file_path = 'C:/Users/Administrator/Anaconda3/envs/temp/EmotionEmbedding/emotion8_finalcode/'
em_layername = file_path+"embeddinglayer/layer_%s.pickle"%name # keras embedding layer name 
glove_filename = file_path+"glovematrix/glovemat_%s.pickle"%name  # save glove embedding matrix 
word_indexname = file_path+"wordindex/wordindex_%s.pickle"%name  # save glove embedding matrix 
roc_datapath = file_path+"data/rocdata/rocstories_sentword_lemma_31m_final.csv"


finalresult = file_path+"finalresult/result%s/result_%s.csv"%(num,name)  ## roc emotion mapping result path 
tweetlabel_datapath = file_path+'data/tweet_labelwords'

In [17]:
def all_files_load():
    with open(em_layername, 'rb') as rotten_file:
        embeddinglayer = pickle.load(rotten_file)
    
    with open(glove_filename, 'rb') as rotten_file:
        glovematrix = pickle.load(rotten_file)
    
    with open(word_indexname, 'rb') as rotten_file:
        wordindex = pickle.load(rotten_file)
        
    roc_data = pd.read_csv(roc_datapath,names = ['storyid','sequence','text','sentword'])
    
    return embeddinglayer, glovematrix, wordindex, roc_data

In [18]:
def get_tweet_labelwords():
    with open(tweetlabel_datapath, 'rb') as rotten_file:
        tweet_labeldict = pickle.load(rotten_file)
        
    return tweet_labeldict

In [19]:
def get_tweetvectormatrix(tweet_labelwordlist, finaldic):
    # get tweet data vector 
    final_tweet_labellist = []
    wordvec = np.zeros(300,)
    for word in tweet_labelwordlist:
        try :
            wordvec = np.vstack([wordvec,finaldic[word]])
            final_tweet_labellist.append(word)
        except KeyError:
            print(word,'is not in embedding matrix')
    
    wordvec = wordvec[1:]
    tweet_vec_mat = wordvec
    return tweet_vec_mat, final_tweet_labellist

In [20]:
if __name__ == "__main__":
    embeddinglayer, glovematrix, wordindex, roc_data = all_files_load()
    tweet_labeldict = get_tweet_labelwords()
    tweet_labelwordlist =  list(set(sum(list(tweet_labeldict.values()), [])))
    
    numlist = list(range(8))
    newdic = dict(zip(numlist,tweet_labeldict.values()))
    tweet_emo_final_dic = {}
    for key,emolist in newdic.items():
        for emo in emolist:
            tweet_emo_final_dic[emo] = key

    
    # dictionary = word : wordvector
    global finaldic
    finaldic = {w:embeddinglayer[idx] for w, idx in wordindex.items()}
    
    global tweet_vec_mat
    tweet_vec_mat, final_tweet_labellist = get_tweetvectormatrix(tweet_labelwordlist,finaldic)
    # roc_data['sentword_vector'] = roc_data['sentword'].apply(get_rocstory_vector)
    
#     tweet_vec_mat = tweet_vec_mat
    
    

enjoyful is not in embedding matrix
gleeful is not in embedding matrix
wrathful is not in embedding matrix
disburned is not in embedding matrix


In [21]:
embeddinglayer.shape

(83204, 300)

# emotion labeling using cosine similarity

In [22]:
def get_rocstory_vector(inputword):
    try :
        inputwordvec = finaldic[inputword]
        cosine_similarity = linear_kernel([inputwordvec], tweet_vec_mat)
        
        simvalue = cosine_similarity[0]
        simvalue_windex = list(enumerate(simvalue))
        
        #print(simvalue_windex)
        
        sim_scores = sorted(simvalue_windex, key=lambda x: x[1], reverse=True)
        sim_scores_top1 = sim_scores[0]
        emotion = final_tweet_labellist[sim_scores_top1[0]]
        emotionlabel = tweet_emo_final_dic[emotion]
        
        sim_scores_top_5 = sim_scores[0:5]
        sim_scores_top_5_index = [i[0] for i in sim_scores_top_5]
        emotionlist = [final_tweet_labellist[i] for i in sim_scores_top_5_index]
        
        return emotionlist, emotionlabel

    except KeyError:
        return 0, None

In [23]:
if __name__ == "__main__": 
    
    roc_data['sentword_vector'],roc_data['emotionlabel'] = zip(*roc_data['sentword'].apply(get_rocstory_vector))
    roc_data_dropna = roc_data.dropna(subset = ['emotionlabel'])
    roc_data_dropna.to_csv(finalresult)

# result

In [25]:
roc_data_dropna

,storyid,sequence,text,sentword,sentword_vector,emotionlabel
4,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,sentence4,he stopped going to burger places and started ...,stopped,"[relief, fear, worried, angry, surprised]",5
5,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,sentence5,after a few weeks he started to feel much better,better,"[happy, relief, worry, interest, interested]",4
6,0beabab2-fb49-460e-a6e6-f35a202e3348,storytitle,frustration,frustration,"[anger, sadness, disgust, despair, rage]",0
7,0beabab2-fb49-460e-a6e6-f35a202e3348,sentence1,tom had a very short temper,temper,"[anger, rage, angry, hatred, enthusiasm]",0
8,0beabab2-fb49-460e-a6e6-f35a202e3348,sentence2,one day a guest made him very angry,angry,"[angry, annoyed, irritated, anger, rage]",0
11,0beabab2-fb49-460e-a6e6-f35a202e3348,sentence5,tom sat on his couch filled with regret about ...,regret,"[sorrow, sadness, grief, anger, sad]",6
13,87da1a22-df0b-410c-b186-439700b70ba6,sentence1,marcus needed clothing for a business casual e...,casual,"[curious, cheerful, enthusiasm, enthusiastic, ...",1
14,87da1a22-df0b-410c-b186-439700b70ba6,sentence2,all of his clothes were either too formal or t...,casual,"[curious, cheerful, enthusiasm, enthusiastic, ...",1
16,87da1a22-df0b-410c-b186-439700b70ba6,sentence4,the pair he bought fit him perfectly,perfectly,"[happy, cheerful, happiness, joy, amazed]",4
17,87da1a22-df0b-410c-b186-439700b70ba6,sentence5,marcus was happy to have the right clothes for...,happy,"[happy, happiness, joy, joyful, joyous]",4


In [27]:
roc_data_dropna['emotionlabel'].value_counts()

4    31371
6    27245
3    22480
0    19711
7    13201
1    13024
5     6769
2     3251
Name: emotionlabel, dtype: int64

In [29]:
### check
roc_data_dropna[roc_data_dropna['emotionlabel']==5]

,storyid,sequence,text,sentword,sentword_vector,emotionlabel
4,8bbe6d11-1e2e-413c-bf81-eaea05f4f1bd,sentence4,he stopped going to burger places and started ...,stopped,"[relief, fear, worried, angry, surprised]",5
109,c50cf35d-564d-40c9-8784-80b4fdec9d16,sentence1,sally had a root canal this morning as she had...,damaged,"[relief, angry, fear, worried, panic]",5
155,ba8e1ad2-ce23-4bae-8b93-9f9ab9053616,sentence5,i started taking medications to combat the sym...,combat,"[relief, fear, involvement, relieved, hatred]",5
184,8d6d55d1-1123-453c-b9dc-6fcacfa72ad2,sentence4,he called for tech support but they couldnt he...,support,"[relief, interest, involvement, enthusiasm, an...",5
428,ad18e260-da97-4a0b-a4db-812cd15cdffa,sentence2,one day he stopped coming to class,stopped,"[relief, fear, worried, angry, surprised]",5
441,c100093b-ade6-4a68-98ff-bd686584fa63,sentence3,but she could reach very few,reach,"[relief, desperate, interest, eager, happy]",5
443,c100093b-ade6-4a68-98ff-bd686584fa63,sentence5,anna wished she had been able to reach more ap...,reach,"[relief, desperate, interest, eager, happy]",5
466,860db75c-57a5-4d92-bae2-485c87a0bddc,sentence4,finally he was prepared to participate in a ma...,prepared,"[relief, eager, surprised, anxious, afraid]",5
473,f14b45d1-17bc-46e7-b5e3-4edbbcb1cffa,sentence5,carly was there giving billy the look that say...,giving,"[relief, fear, interest, interested, wary]",5
516,e78d1d3f-c5b8-4cfb-af59-7fd769dbbba4,storytitle,charlie helps paint,help,"[relief, eager, interest, desperate, fear]",5


# (ignore) Analysispart

In [218]:
numlist = list(range(8))
newdic = dict(zip(numlist,tweet_labeldict.values()))
tweet_emo_final_dic = {}
for key,emolist in newdic.items():
    for emo in emolist:
        tweet_emo_final_dic[emo] = key

In [222]:
newdic

{0: ['anger',
  'irritation',
  'mad',
  'hatred',
  'hateful',
  'madden',
  'fury',
  'annoyed',
  'irritated',
  'angry',
  'rage',
  'wrathful',
  'hate',
  'angrytweet',
  'hateit',
  'irritating'],
 1: ['disgust',
  'repulsion',
  'loathsome',
  'dislike',
  'loathe',
  'disgusting',
  'repulse'],
 2: ['worry',
  'fear',
  'scared',
  'worried',
  'panic',
  'anxious',
  'dreadful',
  'alarmed',
  'wary',
  'nervous',
  'afraid',
  'horrified',
  'worrying'],
 3: ['guilt', 'remorse', 'blameworthy', 'guilty', 'remorseful', 'blamed'],
 4: ['happiness',
  'joy',
  'cheerful',
  'enjoyful',
  'happy',
  'joyful',
  'gleeful',
  'joyous',
  'happytweet',
  'sohappy',
  'happydays'],
 5: ['relief', 'disburned', 'relieved'],
 6: ['sadness',
  'despair',
  'grievous',
  'grief',
  'sad',
  'gloomy',
  'grieving',
  'desperate',
  'sorrow',
  'sorrowful',
  'sadden',
  'weepy',
  'sadtweet',
  'sosad'],
 7: ['surprise',
  'astonishment',
  'amazed',
  'astonished',
  'surprised',
  'start